<h1>Speech T5 Fine-Tuning<h1>

# **Models Trial**
Different trials to clone/0-shot/generate speech from text with different models.

In [ ]:
from transformers import pipeline
from datasets import load_dataset
import soundfile as sf
import torch

synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = synthesiser("Hello, my dog is cooler than you!", forward_params={"speaker_embeddings": speaker_embedding})

sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])

Device set to use cpu


In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

inputs = processor(text="Hello, my dog is cute.", return_tensors="pt")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

sf.write("speech.wav", speech.numpy(), samplerate=16000)

In [ ]:
! pip install git+https://github.com/huggingface/parler-tts.git

  Cloning https://github.com/huggingface/parler-tts.git to /tmp/pip-req-build-y1r3bw1c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/parler-tts.git /tmp/pip-req-build-y1r3bw1c
  Resolved https://github.com/huggingface/parler-tts.git to commit d108732cd57788ec86bc857d99a6cabd66663d68
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/descriptinc/audiotools to /tmp/pip-install-5mfgfi1k/descript-audiotools_cad4910c78f44acaaa80144975873de7
  Running command git clone --filter=blob:none --quiet https://github.com/descriptinc/audiotools /tmp/pip-install-5mfgfi1k/descript-audiotools_cad4910c78f44acaaa80144975873de7
  Resolved https://github.com/descriptinc/audiotools to commit 348ebf2034ce24e2a91a553e3171cb00c0c71678
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.4 MB/s eta 0:00:00
  Prep

In [ ]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-large-v1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-large-v1")

prompt = "Hey, how are you doing today?"
description = "A female speaker delivers a slightly expressive and animated speech with a moderate speed and pitch. The recording is of very high quality, with the speaker's voice sounding clear and very close up."

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()
sf.write("parler_tts_out.wav", audio_arr, model.config.sampling_rate)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/7.72k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/94.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.35G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-large-v1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-large-v1")

prompt = "Hey, how are you doing today?"
description = "Jon's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()
sf.write("parler_tts_out.wav", audio_arr, model.config.sampling_rate)

  "_name_or_path": "google/flan-t5-xl",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 5120,
  "d_kv": 64,
  "d_model": 2048,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 32,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length":

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
! pip install TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 44.0 MB/s et

In [ ]:
! pip install --upgrade transformers accelerate

In [ ]:
from transformers import VitsModel, AutoTokenizer
import torch

model = VitsModel.from_pretrained("facebook/mms-tts-ron")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-ron")

text = "Cred că ar trebui să verificăm încă o dată codul."
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    output = model(**inputs).waveform

In [ ]:
from IPython.display import Audio

Audio(output, rate=model.config.sampling_rate)

In [ ]:
! pip install --upgrade pip
! pip install --upgrade transformers sentencepiece
! pip install datasets
! pip install speechbrain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 52.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and pla

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
import torch
import soundfile as sf
from datasets import load_dataset

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

inputs = processor(text="Hello, my dog is cute.", return_tensors="pt")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

sf.write("speech.wav", speech.numpy(), samplerate=16000)

KeyboardInterrupt: 

In [ ]:
import os
import torch
import torchaudio
import numpy as np
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
device = "cuda" if torch.cuda.is_available() else "cpu"

speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name)
)

def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(waveform.to(device))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

wav_path = "1_vad_right_channel_4313_1728907904.22973_16k.wav"
waveform, sr = torchaudio.load(wav_path)

if sr != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
    waveform = resampler(waveform)

if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

embedding = create_speaker_embedding(waveform)
print("Speaker embedding shape:", embedding.shape)

np.save("speaker_embedding.npy", embedding)
print("Speaker embedding saved as 'speaker_embedding.npy'")

<ipython-input-3-23c3b38c6c98>:5: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier


hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

Speaker embedding shape: (512,)
Speaker embedding saved as 'speaker_embedding.npy'


In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
import torch
import soundfile as sf
import numpy as np

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

inputs = processor(text="Bună ziua, cu ce vă ajut?", return_tensors="pt")

speaker_embedding = np.load("speaker_embedding.npy")
speaker_embeddings = torch.tensor(speaker_embedding).unsqueeze(0)

with torch.no_grad():
    speech = model.generate_speech(
        inputs["input_ids"],
        speaker_embeddings=speaker_embeddings,
        vocoder=vocoder
    )

sf.write("speech.wav", speech.numpy(), samplerate=16000)
print("Audio generated with custom speaker embedding – saved as 'speech.wav'")

Audio generated with custom speaker embedding – saved as 'speech.wav'


# **Vocabulary**
SpeechT5 Tokenizer check.

In [ ]:
from transformers import SpeechT5Tokenizer

tokenizer = SpeechT5Tokenizer.from_pretrained("microsoft/speecht5_tts")

vocab = tokenizer.get_vocab()

sorted_vocab = sorted(vocab.items(), key=lambda item: item[1])

for token, index in sorted_vocab:
    print(f"{index}: {token}")

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

0: <s>
1: <pad>
2: </s>
3: <unk>
4: ▁
5: e
6: t
7: a
8: o
9: n
10: i
11: h
12: s
13: r
14: d
15: l
16: u
17: c
18: m
19: f
20: w
21: g
22: y
23: ,
24: p
25: b
26: .
27: v
28: k
29: "
30: I
31: '
32: T
33: A
34: S
35: H
36: ;
37: x
38: W
39: -
40: B
41: ?
42: C
43: M
44: !
45: q
46: j
47: E
48: N
49: P
50: O
51: D
52: L
53: G
54: R
55: F
56: Y
57: z
58: J
59: :
60: K
61: U
62: V
63: )
64: (
65: Q
66: Z
67: ]
68: [
69: X
70: —
71: /
72: æ
73: é
74: {
75: }
76: ê
77: œ
78: ̄
79: <mask>
80: <ctc_blank>


# **Replace Diacritics**
Main function to replace diacritics and normalize text for SpeechT5.

In [ ]:
import pandas as pd

df = pd.read_csv("train_labels.csv", encoding="utf-8-sig")
all_text = " ".join(df["transcript"].dropna().astype(str))
unique_chars = sorted(set(all_text))
print("".join(unique_chars))

 !"#',-.0123456789:?@ABCDEFGHIJKLMNOPQRSTUVWXZ_abcdefghijklmnopqrstuvwxyz


In [ ]:
import pandas as pd

df = pd.read_csv("Labels.csv", encoding="utf-8-sig")

replacements = [
    ("ă", "a"), ("â", "a"), ("î", "i"), ("ș", "s"), ("ț", "t"),
    ("Ă", "A"), ("Â", "A"), ("Î", "I"), ("Ș", "S"), ("Ț", "T"),
    ("ş", "s"), ("ţ", "t"), ("Ş", "S"), ("Ţ", "T"),
    ("á", "a"), ("é", "e"), ("í", "i"), ("ò", "o"), ("ü", "u"),
    ("É", "E"), ("ë", "e"), ("ç", "c"), ("à", "a"), ("ï", "i"), ("ö", "o"),
    ("ć", "c"), ("č", "c"), ("Đ", "D"), ("Š", "S")
]

def cleanup_text(text):
    if not isinstance(text, str):
        return text
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

df["transcript"] = df["transcript"].apply(cleanup_text)

df.to_csv("Labels_normalized.csv", index=False, encoding="utf-8-sig")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('Labels_normalized.csv')

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_df.to_csv('train_labels.csv', index=False)
test_df.to_csv('test_labels.csv', index=False)

print("Split:")
print(f"Train: {len(train_df)} rows")
print(f"Test: {len(test_df)} rows")

Split completat:
Train: 1784 rânduri
Test: 446 rânduri


# **Create Dataloaders**
Function to process data and create dataloaders.

In [ ]:
! pip install speechbrain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import zipfile
import os

os.makedirs("audio", exist_ok=True)

with zipfile.ZipFile("audio.zip", "r") as zip_ref:
    zip_ref.extractall("audio")

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, get_scheduler
from tqdm import tqdm
from torch.optim import AdamW
from speechbrain.pretrained import EncoderClassifier

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_encoder = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name)
)

speaker_encoder.to(device)

class SpeechT5TTSDataset(Dataset):
    def __init__(self, csv_path, audio_dir, processor, speaker_encoder):
        self.data = pd.read_csv(csv_path)
        self.audio_dir = audio_dir
        self.processor = processor
        self.speaker_encoder = speaker_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio_path = os.path.join(self.audio_dir, self.data.iloc[idx]["audio_file"])
        text = self.data.iloc[idx]["transcript"]

        # Load and preprocess audio
        waveform, sr = torchaudio.load(audio_path)
        if sr != 16000:
            waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # convert to numpy for processor & SpeechBrain
        waveform_np = waveform.squeeze(0).numpy()

        # Generate speaker embedding
        with torch.no_grad():
            speaker_embedding = self.speaker_encoder.encode_batch(waveform)
            speaker_embedding = torch.nn.functional.normalize(speaker_embedding, dim=2)
            speaker_embedding = speaker_embedding.squeeze(0)

        # Use processor for text and audio target labels
        inputs = self.processor(
            text=text,
            audio_target=waveform_np,
            sampling_rate=16000,
            return_attention_mask=False
        )

        # Convert to tensors
        input_ids = inputs["input_ids"]
        if isinstance(input_ids, list):
            input_ids = torch.tensor(input_ids, dtype=torch.long)

        labels = inputs["labels"]
        if isinstance(labels, list):
            labels = torch.tensor(labels, dtype=torch.float32)

        if labels.ndim == 3 and labels.shape[0] == 1:
            labels = labels.squeeze(0)

        if labels.ndim != 2:
            raise ValueError(f"Label tensor should be 2D, but is {labels.shape}")

        mel_bins = labels.shape[1]
        if mel_bins < 80:
            labels = torch.nn.functional.pad(labels, (0, 80 - mel_bins), value=0.0)
        elif mel_bins > 80:
            labels = labels[:, :80]

        return {
            "input_ids": input_ids,
            "labels": labels,
            "speaker_embeddings": speaker_embedding
        }

def collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]

    labels = []
    for item in batch:
        l = item["labels"]
        if isinstance(l, torch.Tensor):
            l = l.cpu().numpy()
        labels.append(l.astype(np.float32))

    speaker_embeddings = [item["speaker_embeddings"] for item in batch]

    input_padded = processor.tokenizer.pad(
        {"input_ids": input_ids},
        padding=True,
        return_tensors="pt"
    )

    label_padded = torch.nn.utils.rnn.pad_sequence(
        [torch.tensor(l, dtype=torch.float32) for l in labels],
        batch_first=True,
        padding_value=-100.0
    )

    reduction_factor = getattr(model.config, "reduction_factor", 2)
    if reduction_factor > 1:
        max_len = label_padded.shape[1]
        max_len = max_len - (max_len % reduction_factor)
        label_padded = label_padded[:, :max_len]

    return {
        "input_ids": input_padded["input_ids"],
        "labels": label_padded,
        "speaker_embeddings": torch.stack(speaker_embeddings)
    }

# Load processor and model
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)

train_csv = "train.csv"
test_csv = "test.csv"
audio_dir = "./audio"

train_dataset = SpeechT5TTSDataset(train_csv, audio_dir, processor, speaker_encoder)
test_dataset = SpeechT5TTSDataset(test_csv, audio_dir, processor, speaker_encoder)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Using symlink found at '/tmp/speechbrain/spkrec-xvect-voxceleb/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Collecting files (or symlinks) for pretraining in /tmp/speechbrain/spkrec-xvect-voxceleb.
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Using symlink found at '/tmp/speechbrain/spkrec-xvect-voxceleb/embedding_model.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /tmp/speechbrain/spkrec-xvect-voxceleb/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Using symlink found at '/tmp/speechbrain/spkrec-xvect-voxceleb/mean_var_norm_emb.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /tmp/speechbrain/spkrec-xvect-voxceleb/mean_var_norm_emb.ck

In [ ]:
def inspect_batch(loader, name="Loader"):
    print(f"\n\n================ Inspecting {name} =================")
    sample_batch = next(iter(loader))

    print("\n=== Batch Shape Details ===")
    print(f"input_ids shape        : {sample_batch['input_ids'].shape}")
    print(f"labels shape           : {sample_batch['labels'].shape}")
    print(f"speaker_embeddings shape: {sample_batch['speaker_embeddings'].shape}")

    # Check mel bins
    num_mel_bins = sample_batch["labels"].shape[-1]
    print(f"\nDetected mel_bins dimension: {num_mel_bins}")
    if num_mel_bins != 80:
        print("WARNING: Expected 80 mel bins but got something else!")
    else:
        print("Success: All spectrograms have 80 mel bins.")

    # Inspect each sample in batch
    print("\n=== Per-sample Information ===")
    for i in range(sample_batch['input_ids'].shape[0]):
        input_ids = sample_batch['input_ids'][i]
        decoded_text = processor.tokenizer.decode(
            input_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )

        print(f"\n--- Example {i + 1} ---")
        print(f"input_ids length       : {input_ids.shape[0]}")
        print(f"spectrogram frames     : {sample_batch['labels'][i].shape[0]}")
        print(f"mel bins per frame     : {sample_batch['labels'][i].shape[1]}")
        print(f"speaker embedding shape: {sample_batch['speaker_embeddings'][i].shape}")
        print(f"first input_ids        : {input_ids[:10].tolist()}")
        print(f"first label frame      : {sample_batch['labels'][i][0][:5].tolist()}")
        print(f"speaker embedding (5)  : {sample_batch['speaker_embeddings'][i][:5].tolist()}")
        print(f"Decoded text           : \"{decoded_text}\"")

# Run inspection on both loaders
inspect_batch(train_loader, name="Train Loader")
inspect_batch(test_loader, name="Test Loader")



================ Inspecting Train Loader =================

=== Batch Shape Details ===
input_ids shape        : torch.Size([2, 22])
labels shape           : torch.Size([2, 626, 80])
speaker_embeddings shape: torch.Size([2, 1, 512])

Detected mel_bins dimension: 80
Success: All spectrograms have 80 mel bins.

=== Per-sample Information ===

--- Example 1 ---
input_ids length       : 22
spectrogram frames     : 626
mel bins per frame     : 80
speaker embedding shape: torch.Size([1, 512])
first input_ids        : [4, 48, 16, 4, 17, 13, 5, 14, 4, 17]
first label frame      : [-2.1019859313964844, -2.196446418762207, -1.9874480962753296, -1.9570350646972656, -2.167734146118164]
speaker embedding (5)  : [[-0.06416033208370209, 0.02709406055510044, 0.04900325834751129, 0.011934646405279636, 0.03597863391041756, 0.005255556665360928, -0.0811418816447258, 0.03402790054678917, 0.024268660694360733, 0.017456181347370148, -0.08652994781732559, -0.07032079249620438, 0.053686462342739105, 0.01916

# **Fine-Tuning**
Fine-Tuning pipeline of SpeechT5 with different datasets.

In [ ]:
%%writefile fine_tune_speecht5.py
import os
import random
import pandas as pd
import numpy as np
import torch
import torchaudio
import matplotlib.pyplot as plt
import csv
from torch.utils.data import Dataset, DataLoader
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, get_scheduler
from tqdm import tqdm
from torch.optim import AdamW
from speechbrain.pretrained import EncoderClassifier

# Fix random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_encoder = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name)
)

speaker_encoder.to(device)

class SpeechT5TTSDataset(Dataset):
    def __init__(self, csv_path, audio_dir, processor, speaker_encoder):
        self.data = pd.read_csv(csv_path)
        self.audio_dir = audio_dir
        self.processor = processor
        self.speaker_encoder = speaker_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio_path = os.path.join(self.audio_dir, self.data.iloc[idx]["audio_file"])
        text = self.data.iloc[idx]["transcript"]

        # Load and preprocess audio
        waveform, sr = torchaudio.load(audio_path)
        if sr != 16000:
            waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # convert to numpy for processor & SpeechBrain
        waveform_np = waveform.squeeze(0).numpy()

        waveform_np = waveform_np / np.abs(waveform_np).max()

        # Generate speaker embedding
        with torch.no_grad():
            speaker_embedding = self.speaker_encoder.encode_batch(waveform)
            speaker_embedding = torch.nn.functional.normalize(speaker_embedding, dim=2)
            speaker_embedding = speaker_embedding.squeeze(0).squeeze(0)

        # Use processor for text and audio target labels
        inputs = self.processor(
            text=text,
            audio_target=waveform_np,
            sampling_rate=16000,
            return_attention_mask=False
        )

        # Convert to tensors
        input_ids = inputs["input_ids"]
        if isinstance(input_ids, list):
            input_ids = torch.tensor(input_ids, dtype=torch.long)

        labels = inputs["labels"]
        if isinstance(labels, list):
            labels = torch.tensor(labels, dtype=torch.float32)

        if labels.ndim == 3 and labels.shape[0] == 1:
            labels = labels.squeeze(0)

        if labels.ndim != 2:
            raise ValueError(f"Label tensor should be 2D, but is {labels.shape}")

        mel_bins = labels.shape[1]
        if mel_bins < 80:
            labels = torch.nn.functional.pad(labels, (0, 80 - mel_bins), value=0.0)
        elif mel_bins > 80:
            labels = labels[:, :80]

        return {
            "input_ids": input_ids,
            "labels": labels,
            "speaker_embeddings": speaker_embedding
        }

def collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    labels = []
    for item in batch:
        l = item["labels"]
        if isinstance(l, torch.Tensor):
            l = l.cpu().numpy()
        labels.append(l.astype(np.float32))

    speaker_embeddings = [item["speaker_embeddings"] for item in batch]

    # Pad input_ids and attention_mask
    input_padded = processor.tokenizer.pad(
        {"input_ids": input_ids},
        padding=True,
        return_tensors="pt"
    )

    # Create attention_mask from the padded input_ids
    attention_mask = input_padded["attention_mask"]

    # Pad the labels to ensure proper length
    label_padded = torch.nn.utils.rnn.pad_sequence(
        [torch.tensor(l, dtype=torch.float32) for l in labels],
        batch_first=True,
        padding_value=-100.0
    )

    # Adjust label length based on model's reduction factor
    reduction_factor = getattr(model.config, "reduction_factor", 2)
    if reduction_factor > 1:
        max_len = label_padded.shape[1]
        max_len = max_len - (max_len % reduction_factor)
        label_padded = label_padded[:, :max_len]

    return {
        "input_ids": input_padded["input_ids"],
        "attention_mask": attention_mask,
        "labels": label_padded,
        "speaker_embeddings": torch.stack(speaker_embeddings)
    }

# Early stopping
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best = float('inf')
        self.counter = 0

    def step(self, val_loss):
        if val_loss < self.best - self.min_delta:
            self.best = val_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Train function
def train_one_epoch(model, dataloader):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader, desc="Training"):
        # Transfer all tensors to the GPU
        for key in batch:
            batch[key] = batch[key].to(device)

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
            speaker_embeddings=batch["speaker_embeddings"]
        )

        # Loss computation and backward pass
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    return total_loss / len(dataloader)

# Eval function
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            for key in batch:
                batch[key] = batch[key].to(device)

            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=batch["labels"],
                speaker_embeddings=batch["speaker_embeddings"]
            )

            loss = outputs.loss
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

# Output & metrics
save_dir = "speecht5_finetuned"
metrics_csv = "tts_epoch_metrics.csv"
os.makedirs(save_dir, exist_ok=True)

# Load processor & model
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)

# Load Data
train_csv = "train_common_voices20_normalized.csv"
test_csv = "test_common_voices20_normalized.csv"
audio_dir = "./common_voices20_audio_tts"

train_dataset = SpeechT5TTSDataset(train_csv, audio_dir, processor, speaker_encoder)
test_dataset = SpeechT5TTSDataset(test_csv, audio_dir, processor, speaker_encoder)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Optimizer & scheduler
lr = 5e-5
num_epochs = 40
optimizer = AdamW(model.parameters(), lr=lr)
num_training_steps = len(train_loader) * num_epochs
num_warmup_steps = int(0.1 * num_training_steps)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

# Train loop
train_losses, val_losses = [], []
best_model = None
best_val = float("inf")
maes, mses = [], []

early_stopper = EarlyStopping(patience=5)

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    train_loss = train_one_epoch(model, train_loader)
    val_loss = evaluate(model, test_loader)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    with open(metrics_csv, "a", newline="") as f:
        writer = csv.writer(f)
        if epoch == 0:
            writer.writerow(["epoch", "train_loss", "val_loss", "lr"])
        writer.writerow([epoch+1, train_loss, val_loss, scheduler.get_last_lr()[0]])

    if val_loss < best_val:
        best_val = val_loss
        best_model = model
        model.save_pretrained(save_dir)
        processor.save_pretrained(save_dir)
        print("✅ New best model saved.")

    if early_stopper.step(val_loss):
        print("⛔ Early stopping triggered.")
        break

# Plot losses
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.grid()
plt.legend()
plt.savefig("tts_loss_plot.png")
plt.close()

print("🎉 Training complete!")

Writing fine_tune_speecht5.py


In [ ]:
%%writefile fine_tune_speecht5.py
import os
import random
import pandas as pd
import numpy as np
import torch
import torchaudio
import matplotlib.pyplot as plt
import csv
from torch.utils.data import Dataset, DataLoader
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, get_scheduler
from tqdm import tqdm
from torch.optim import AdamW
from speechbrain.pretrained import EncoderClassifier

# Fix random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_encoder = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name)
)

speaker_encoder.to(device)

class SpeechT5TTSDataset(Dataset):
    def __init__(self, csv_path, audio_dir, processor, speaker_encoder):
        self.data = pd.read_csv(csv_path)
        self.audio_dir = audio_dir
        self.processor = processor
        self.speaker_encoder = speaker_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio_path = os.path.join(self.audio_dir, self.data.iloc[idx]["audio_file"])
        text = self.data.iloc[idx]["transcript"]

        # Load and preprocess audio
        waveform, sr = torchaudio.load(audio_path)
        if sr != 16000:
            waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # convert to numpy for processor & SpeechBrain
        waveform_np = waveform.squeeze(0).numpy()

        waveform_np = waveform_np / np.abs(waveform_np).max()

        # Generate speaker embedding
        with torch.no_grad():
            speaker_embedding = self.speaker_encoder.encode_batch(waveform)
            speaker_embedding = torch.nn.functional.normalize(speaker_embedding, dim=2)
            speaker_embedding = speaker_embedding.squeeze(0).squeeze(0)

        # Use processor for text and audio target labels
        inputs = self.processor(
            text=text,
            audio_target=waveform_np,
            sampling_rate=16000,
            return_attention_mask=False
        )

        # Convert to tensors
        input_ids = inputs["input_ids"]
        if isinstance(input_ids, list):
            input_ids = torch.tensor(input_ids, dtype=torch.long)

        labels = inputs["labels"]
        if isinstance(labels, list):
            labels = torch.tensor(labels, dtype=torch.float32)

        if labels.ndim == 3 and labels.shape[0] == 1:
            labels = labels.squeeze(0)

        if labels.ndim != 2:
            raise ValueError(f"Label tensor should be 2D, but is {labels.shape}")

        mel_bins = labels.shape[1]
        if mel_bins < 80:
            labels = torch.nn.functional.pad(labels, (0, 80 - mel_bins), value=0.0)
        elif mel_bins > 80:
            labels = labels[:, :80]

        return {
            "input_ids": input_ids,
            "labels": labels,
            "speaker_embeddings": speaker_embedding
        }

def collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    labels = []
    for item in batch:
        l = item["labels"]
        if isinstance(l, torch.Tensor):
            l = l.cpu().numpy()
        labels.append(l.astype(np.float32))

    speaker_embeddings = [item["speaker_embeddings"] for item in batch]

    # Pad input_ids and attention_mask
    input_padded = processor.tokenizer.pad(
        {"input_ids": input_ids},
        padding=True,
        return_tensors="pt"
    )

    # Create attention_mask from the padded input_ids
    attention_mask = input_padded["attention_mask"]

    # Pad the labels to ensure proper length
    label_padded = torch.nn.utils.rnn.pad_sequence(
        [torch.tensor(l, dtype=torch.float32) for l in labels],
        batch_first=True,
        padding_value=-100.0
    )

    # Adjust label length based on model's reduction factor
    reduction_factor = getattr(model.config, "reduction_factor", 2)
    if reduction_factor > 1:
        max_len = label_padded.shape[1]
        max_len = max_len - (max_len % reduction_factor)
        label_padded = label_padded[:, :max_len]

    return {
        "input_ids": input_padded["input_ids"],
        "attention_mask": attention_mask,
        "labels": label_padded,
        "speaker_embeddings": torch.stack(speaker_embeddings)
    }

# Early stopping
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best = float('inf')
        self.counter = 0

    def step(self, val_loss):
        if val_loss < self.best - self.min_delta:
            self.best = val_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Train function
def train_one_epoch(model, dataloader):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader, desc="Training"):
        # Transfer all tensors to the GPU
        for key in batch:
            batch[key] = batch[key].to(device)

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
            speaker_embeddings=batch["speaker_embeddings"]
        )

        # Loss computation and backward pass
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    return total_loss / len(dataloader)

# Eval function
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            for key in batch:
                batch[key] = batch[key].to(device)

            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=batch["labels"],
                speaker_embeddings=batch["speaker_embeddings"]
            )

            loss = outputs.loss
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

# Output & metrics
save_dir = "speecht5_finetuned_od"
metrics_csv = "tts_epoch_metrics.csv"
os.makedirs(save_dir, exist_ok=True)

# Load processor & model
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)

# Load Data
train_csv = "train_labels.csv"
test_csv = "test_labels.csv"
audio_dir = "./Audio"

train_dataset = SpeechT5TTSDataset(train_csv, audio_dir, processor, speaker_encoder)
test_dataset = SpeechT5TTSDataset(test_csv, audio_dir, processor, speaker_encoder)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Optimizer & scheduler
lr = 5e-5
num_epochs = 40
optimizer = AdamW(model.parameters(), lr=lr)
num_training_steps = len(train_loader) * num_epochs
num_warmup_steps = int(0.1 * num_training_steps)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

# Train loop
train_losses, val_losses = [], []
best_model = None
best_val = float("inf")
maes, mses = [], []

early_stopper = EarlyStopping(patience=3)

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    train_loss = train_one_epoch(model, train_loader)
    val_loss = evaluate(model, test_loader)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    with open(metrics_csv, "a", newline="") as f:
        writer = csv.writer(f)
        if epoch == 0:
            writer.writerow(["epoch", "train_loss", "val_loss", "lr"])
        writer.writerow([epoch+1, train_loss, val_loss, scheduler.get_last_lr()[0]])

    if val_loss < best_val:
        best_val = val_loss
        best_model = model
        model.save_pretrained(save_dir)
        processor.save_pretrained(save_dir)
        print("✅ New best model saved.")

    if early_stopper.step(val_loss):
        print("⛔ Early stopping triggered.")
        break

# Plot losses
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.grid()
plt.legend()
plt.savefig("tts_loss_plot.png")
plt.close()

print("🎉 Training complete!")

Overwriting fine_tune_speecht5.py


**Specific for a voice**

In [ ]:
%%writefile fine_tune_speecht5.py
import os
import random
import pandas as pd
import numpy as np
import torch
import torchaudio
import matplotlib.pyplot as plt
import csv
from torch.utils.data import Dataset, DataLoader
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, get_scheduler
from tqdm import tqdm
from torch.optim import AdamW
from speechbrain.pretrained import EncoderClassifier

# Fix random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
speaker_encoder = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name)
)

speaker_encoder.to(device)

class SpeechT5TTSDataset(Dataset):
    def __init__(self, csv_path, audio_dir, processor, speaker_encoder):
        self.data = pd.read_csv(csv_path)
        self.audio_dir = audio_dir
        self.processor = processor
        self.speaker_encoder = speaker_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio_path = os.path.join(self.audio_dir, self.data.iloc[idx]["audio_file"])
        text = self.data.iloc[idx]["transcript"]

        # Load and preprocess audio
        waveform, sr = torchaudio.load(audio_path)
        if sr != 16000:
            waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # convert to numpy for processor & SpeechBrain
        waveform_np = waveform.squeeze(0).numpy()

        waveform_np = waveform_np / np.abs(waveform_np).max()

        # Generate speaker embedding
        with torch.no_grad():
            speaker_embedding = self.speaker_encoder.encode_batch(waveform)
            speaker_embedding = torch.nn.functional.normalize(speaker_embedding, dim=2)
            speaker_embedding = speaker_embedding.squeeze(0).squeeze(0)

        # Use processor for text and audio target labels
        inputs = self.processor(
            text=text,
            audio_target=waveform_np,
            sampling_rate=16000,
            return_attention_mask=False
        )

        # Convert to tensors
        input_ids = inputs["input_ids"]
        if isinstance(input_ids, list):
            input_ids = torch.tensor(input_ids, dtype=torch.long)

        labels = inputs["labels"]
        if isinstance(labels, list):
            labels = torch.tensor(labels, dtype=torch.float32)

        if labels.ndim == 3 and labels.shape[0] == 1:
            labels = labels.squeeze(0)

        if labels.ndim != 2:
            raise ValueError(f"Label tensor should be 2D, but is {labels.shape}")

        mel_bins = labels.shape[1]
        if mel_bins < 80:
            labels = torch.nn.functional.pad(labels, (0, 80 - mel_bins), value=0.0)
        elif mel_bins > 80:
            labels = labels[:, :80]

        return {
            "input_ids": input_ids,
            "labels": labels,
            "speaker_embeddings": speaker_embedding
        }

def collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    labels = []
    for item in batch:
        l = item["labels"]
        if isinstance(l, torch.Tensor):
            l = l.cpu().numpy()
        labels.append(l.astype(np.float32))

    speaker_embeddings = [item["speaker_embeddings"] for item in batch]

    # Pad input_ids and attention_mask
    input_padded = processor.tokenizer.pad(
        {"input_ids": input_ids},
        padding=True,
        return_tensors="pt"
    )

    # Create attention_mask from the padded input_ids
    attention_mask = input_padded["attention_mask"]

    # Pad the labels to ensure proper length
    label_padded = torch.nn.utils.rnn.pad_sequence(
        [torch.tensor(l, dtype=torch.float32) for l in labels],
        batch_first=True,
        padding_value=-100.0
    )

    # Adjust label length based on model's reduction factor
    reduction_factor = getattr(model.config, "reduction_factor", 2)
    if reduction_factor > 1:
        max_len = label_padded.shape[1]
        max_len = max_len - (max_len % reduction_factor)
        label_padded = label_padded[:, :max_len]

    return {
        "input_ids": input_padded["input_ids"],
        "attention_mask": attention_mask,
        "labels": label_padded,
        "speaker_embeddings": torch.stack(speaker_embeddings)
    }

# Early stopping
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best = float('inf')
        self.counter = 0

    def step(self, val_loss):
        if val_loss < self.best - self.min_delta:
            self.best = val_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Train function
def train_one_epoch(model, dataloader):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader, desc="Training"):
        # Transfer all tensors to the GPU
        for key in batch:
            batch[key] = batch[key].to(device)

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
            speaker_embeddings=batch["speaker_embeddings"]
        )

        # Loss computation and backward pass
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    return total_loss / len(dataloader)

# Eval function
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0.0
    total_mae = 0.0
    total_mse = 0.0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            for key in batch:
                batch[key] = batch[key].to(device)

            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=batch["labels"],
                speaker_embeddings=batch["speaker_embeddings"]
            )

            loss = outputs.loss
            total_loss += loss.item()

            spectrogram_pred = outputs.spectrogram
            labels = batch["labels"]

            min_len = min(spectrogram_pred.shape[1], labels.shape[1])
            spectrogram_pred = spectrogram_pred[:, :min_len, :]
            labels = labels[:, :min_len, :]

            total_mae += torch.nn.functional.l1_loss(spectrogram_pred, labels).item()
            total_mse += torch.nn.functional.mse_loss(spectrogram_pred, labels).item()

    avg_loss = total_loss / len(dataloader)
    avg_mae = total_mae / len(dataloader)
    avg_mse = total_mse / len(dataloader)

    return avg_loss, avg_mae, avg_mse

# Output & metrics
save_dir = "speecht5_finetuned_cv_od"
metrics_csv = "tts_epoch_metrics.csv"
os.makedirs(save_dir, exist_ok=True)

# Load processor & model
model_path = os.path.abspath("speecht5_finetuned")
processor = SpeechT5Processor.from_pretrained(model_path, local_files_only=True)
model = SpeechT5ForTextToSpeech.from_pretrained(model_path, local_files_only=True).to(device)

# Load Data
train_csv = "train_labels.csv"
test_csv = "test_labels.csv"
audio_dir = "./Audio"

train_dataset = SpeechT5TTSDataset(train_csv, audio_dir, processor, speaker_encoder)
test_dataset = SpeechT5TTSDataset(test_csv, audio_dir, processor, speaker_encoder)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Optimizer & scheduler
lr = 5e-5
num_epochs = 40
optimizer = AdamW(model.parameters(), lr=lr)
num_training_steps = len(train_loader) * num_epochs
num_warmup_steps = int(0.1 * num_training_steps)
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

# Train loop
train_losses, val_losses = [], []
best_model = None
best_val = float("inf")
maes, mses = [], []

early_stopper = EarlyStopping(patience=5)

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    train_loss = train_one_epoch(model, train_loader)
    val_loss, val_mae, val_mse = evaluate(model, test_loader)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    maes.append(val_mae)
    mses.append(val_mse)

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | MAE: {val_mae:.4f} | MSE: {val_mse:.4f}")

    with open(metrics_csv, "a", newline="") as f:
        writer = csv.writer(f)
        if epoch == 0:
            writer.writerow(["epoch", "train_loss", "val_loss", "mae", "mse", "lr"])
        writer.writerow([epoch+1, train_loss, val_loss, val_mae, val_mse, scheduler.get_last_lr()[0]])

    if val_loss < best_val:
        best_val = val_loss
        best_model = model
        model.save_pretrained(save_dir)
        processor.save_pretrained(save_dir)
        print("✅ New best model saved.")

    if early_stopper.step(val_loss):
        print("⛔ Early stopping triggered.")
        break

# Plot losses
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.grid()
plt.legend()
plt.savefig("tts_loss_plot.png")
plt.close()

# Plot MAE and MSE
plt.plot(maes, label="MAE")
plt.plot(mses, label="MSE")
plt.xlabel("Epoch")
plt.ylabel("Error")
plt.title("MAE and MSE over Epochs")
plt.grid()
plt.legend()
plt.savefig("tts_mae_mse_plot.png")
plt.close()

print("🎉 Training complete!")


Writing fine_tune_speecht5.py


# **Generate Audio**
Function to generate audio in .wav format using HifiGAN.

In [ ]:
%%writefile generate_test.py
import torch
import torchaudio
import numpy as np
from transformers import SpeechT5ForTextToSpeech, SpeechT5Processor
from speechbrain.pretrained import EncoderClassifier
import soundfile as sf

# Load the fine-tuned model and processor
processor = SpeechT5Processor.from_pretrained("speecht5_finetuned")
model = SpeechT5ForTextToSpeech.from_pretrained("speecht5_finetuned").to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

# Load the speaker encoder
speaker_encoder = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-xvect-voxceleb",
    run_opts={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    savedir="/tmp/speechbrain/spkrec-xvect-voxceleb"
)

# === User Input ===
text_input = "Salut! Acesta este un test."
voice_sample_path = "common_voices20_audio_tts/common_voice_ro_20349005.wav"

# Load and preprocess the voice sample
waveform, sr = torchaudio.load(voice_sample_path)
if sr != 16000:
    waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)(waveform)
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

# Normalize waveform
waveform = waveform / waveform.abs().max()

# Generate speaker embedding
with torch.no_grad():
    speaker_embedding = speaker_encoder.encode_batch(waveform)
    speaker_embedding = torch.nn.functional.normalize(speaker_embedding, dim=2)
    speaker_embedding = speaker_embedding.squeeze(0).squeeze(0)
    speaker_embedding = speaker_embedding.unsqueeze(0)

# Process the input text
inputs = processor(text=text_input, return_tensors="pt")

# Move everything to the appropriate device
inputs = {k: v.to(model.device) for k, v in inputs.items()}
speaker_embedding = speaker_embedding.to(model.device)

# Generate speech audio
with torch.no_grad():
    generated_speech = model.generate_speech(
        input_ids=inputs["input_ids"],
        speaker_embeddings=speaker_embedding
    )

# Save the generated waveform to a file
sf.write("generated_speech.wav", generated_speech.cpu().numpy(), 16000)

print("✅ Speech generated and saved as 'generated_speech.wav'")

Overwriting generate_test.py


In [1]:
%%writefile generate_test.py
import torch
import torchaudio
import numpy as np
import soundfile as sf
from transformers import (
    SpeechT5Processor,
    SpeechT5ForTextToSpeech,
    SpeechT5HifiGan
)
from speechbrain.inference import EncoderClassifier

# ====== CONFIG ======
TEXT_INPUT = "Salut! Acesta este un test."
VOICE_SAMPLE_PATH = "common_voices20_audio_tts/common_voice_ro_20349005.wav"
OUTPUT_WAV_PATH = "generated_speech.wav"
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

# ====== LOAD MODELS ======
# Load the processor and fine-tuned model
processor = SpeechT5Processor.from_pretrained("speecht5_finetuned")
model = SpeechT5ForTextToSpeech.from_pretrained("speecht5_finetuned").to(DEVICE).eval()

# Load the vocoder and move it to the same device
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(DEVICE)

# Load the speaker encoder
speaker_encoder = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-xvect-voxceleb",
    run_opts={"device": DEVICE},
    savedir="/tmp/speechbrain/spkrec-xvect-voxceleb"
)

# ====== PREPROCESS VOICE SAMPLE ======
waveform, sr = torchaudio.load(VOICE_SAMPLE_PATH)

# Resample if needed
if sr != 16000:
    waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)(waveform)

# Convert to mono if stereo
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

# Normalize audio
waveform = waveform / waveform.abs().max()

# Generate speaker embedding
with torch.no_grad():
    speaker_embedding = speaker_encoder.encode_batch(waveform)
    speaker_embedding = torch.nn.functional.normalize(speaker_embedding, dim=2)
    speaker_embedding = speaker_embedding.squeeze(0).squeeze(0).unsqueeze(0)

# ====== TEXT TO SPEECH INFERENCE ======
# Tokenize the input text
inputs = processor(text=TEXT_INPUT, return_tensors="pt").to(DEVICE)

# Generate speech waveform using the vocoder
with torch.no_grad():
    generated_waveform = model.generate_speech(
        input_ids=inputs["input_ids"],
        speaker_embeddings=speaker_embedding.to(DEVICE),
        vocoder=vocoder
    )

# ====== SAVE TO FILE ======
sf.write(OUTPUT_WAV_PATH, generated_waveform.cpu().numpy(), 16000, subtype="PCM_16")
print(f"✅ Speech generated and saved to '{OUTPUT_WAV_PATH}'")

Writing generate_test.py


In [2]:
%%writefile generate_test.py
import torch
import torchaudio
import numpy as np
import soundfile as sf
from transformers import (
    SpeechT5Processor,
    SpeechT5ForTextToSpeech,
    SpeechT5HifiGan
)
from speechbrain.inference import EncoderClassifier

# ====== CONFIG ======
TEXT_INPUT = "Salut! Acesta este un test."
VOICE_SAMPLE_PATH = "./Audio/3_right_channel_4671_1729844990.28155_16k.wav"
OUTPUT_WAV_PATH = "generated_speech.wav"
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

# ====== LOAD MODELS ======
# Load the processor and fine-tuned model
processor = SpeechT5Processor.from_pretrained("./speecht5_finetuned_cv_od")
model = SpeechT5ForTextToSpeech.from_pretrained("./speecht5_finetuned_cv_od").to(DEVICE).eval()

# Load the vocoder and move it to the same device
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(DEVICE)

# Load the speaker encoder
speaker_encoder = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-xvect-voxceleb",
    run_opts={"device": DEVICE},
    savedir="/tmp/speechbrain/spkrec-xvect-voxceleb"
)

# ====== PREPROCESS VOICE SAMPLE ======
waveform, sr = torchaudio.load(VOICE_SAMPLE_PATH)

# Resample if needed
if sr != 16000:
    waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)(waveform)

# Convert to mono if stereo
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

# Normalize audio
waveform = waveform / waveform.abs().max()

# Generate speaker embedding
with torch.no_grad():
    speaker_embedding = speaker_encoder.encode_batch(waveform)
    speaker_embedding = torch.nn.functional.normalize(speaker_embedding, dim=2)
    speaker_embedding = speaker_embedding.squeeze(0).squeeze(0).unsqueeze(0)

# ====== TEXT TO SPEECH INFERENCE ======
# Tokenize the input text
inputs = processor(text=TEXT_INPUT, return_tensors="pt").to(DEVICE)

# Generate speech waveform using the vocoder
with torch.no_grad():
    generated_waveform = model.generate_speech(
        input_ids=inputs["input_ids"],
        speaker_embeddings=speaker_embedding.to(DEVICE),
        vocoder=vocoder
    )

# ====== SAVE TO FILE ======
sf.write(OUTPUT_WAV_PATH, generated_waveform.cpu().numpy(), 16000, subtype="PCM_16")
print(f"✅ Speech generated and saved to '{OUTPUT_WAV_PATH}'")

Overwriting generate_test.py
